In [26]:
from dataclasses import dataclass
from typing import Literal
import numpy as np
import json
import gurobipy as gp
from tqdm import tqdm

# for reproducibility
np.random.seed(42)

In [27]:
@dataclass
class ScenarioParams:
    # Provided by scale of problem in `data/scales` folder
    num_warehouses: int
    num_goods: int
    num_retailers: int

    # Provided by scenario in `data/scenarios` folder
    num_periods: int
    truck_capacity: int
    truck_daily_rental_cost: int
    truck_rental_period: Literal["daily", "weekly"]

    def __post_init__(self):
        # Derived quantities and synthetic data
        self.good_sizes: np.array = np.random.randint(1, 20, self.num_goods)
        self.holding_costs: np.array = np.random.randint(20, 400, self.num_goods)
        self.shortage_costs = self.holding_costs * np.random.uniform(1, 1.2, size=self.num_goods)
        self.transportation_costs = np.round(np.random.uniform(1, 4, (self.num_warehouses, self.num_retailers, self.num_goods)), 1)
        self.retailer_good_demand = np.round(np.random.randint(40, 100, (self.num_retailers, self.num_goods, self.num_periods)), 1)

        # Black Friday simulation: apply scaling factor of [1.0, 0.7, 0.3, 0.1, 4.0, 2.0, 1.0] to demand profile
        for retailer in range(self.num_retailers):
            for good in range(self.num_goods):
                self.retailer_good_demand[retailer][good] = np.multiply(self.retailer_good_demand[retailer][good], [1.0, 0.7, 0.4, 0.2, 3.0, 1.5, 1.0])

        self.max_trucks: int = int(1.2 * self.good_sizes @ np.sum(self.retailer_good_demand, axis=(0, 2)) / self.num_periods)

In [21]:
with open("../data/scenarios/daily_trucks_high_cost.json", 'r') as f:
    scenario_params = json.load(f)

with open("../data/scales/small.json", "r") as g:
    scale_params = json.load(g)

print(scenario_params)
print(scale_params)

{'num_periods': 7, 'truck_rental_period': 'daily', 'truck_capacity': 10500, 'truck_daily_rental_cost': 300}
{'num_warehouses': 10, 'num_goods': 25, 'num_retailers': 25}


In [ ]:
def make_instance(params=None, scale=None, inst_num=None):
    np.random.seed(inst_num)
    params.num_warehouses = scale['num_warehouses']
    params.num_goods = scale['num_goods']
    params.num_retailers = scale['num_retailers']
    params.num_periods = scale['num_periods']

    params.good_sizes = np.random.randint(1, 20, params.num_goods)
    params.holding_costs = np.random.randint(20, 400, params.num_goods)
    params.shortage_costs = params.holding_costs * np.random.uniform(1, 1.2, size=params.num_goods)
    params.transportation_costs = np.round(np.random.uniform(1, 4, (params.num_warehouses, params.num_retailers, params.num_goods)), 1)
    params.retailer_good_demand = np.round(np.random.randint(40, 100, (params.num_retailers, params.num_goods, params.num_periods)), 1)
    return params

In [ ]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 5,
    'num_goods': 30,
    'num_retailers': 15,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']
params.num_warehouses = scale['num_warehouses']
params.num_goods = scale['num_goods']
params.num_retailers = scale['num_retailers']
params.num_periods = scale['num_periods']

params.good_sizes = np.random.randint(1, 20, params.num_goods)
params.holding_costs = np.random.randint(20, 400, params.num_goods)
params.shortage_costs = params.holding_costs * np.random.uniform(1, 1.2, size=params.num_goods)
params.transportation_costs = np.round(np.random.uniform(1, 4, (params.num_warehouses, params.num_retailers, params.num_goods)), 1)
params.retailer_good_demand = np.round(np.random.randint(40, 100, (params.num_retailers, params.num_goods, params.num_periods)), 1)

In [3]:
def solve_opt_model(
    scenario_parameters: Params = None,
    opt_param_dict: dict = None,
    branch: bool = False
):
    m = gp.Model()
    m.setParam('OutputFlag', 0)
    
    for key, value in opt_param_dict.items():
        m.setParam(key, value)
    
    max_trucks = scenario_parameters.max_trucks
    truck_capacity = scenario_parameters.truck_capacity
    truck_daily_rental_cost = scenario_parameters.truck_daily_rental_cost
    truck_rental_period = scenario_parameters.truck_rental_period
    good_sizes = scenario_parameters.good_sizes
    holding_costs = scenario_parameters.holding_costs
    shortage_costs = scenario_parameters.shortage_costs
    transportation_costs = scenario_parameters.transportation_costs
    retailer_good_demand = scenario_parameters.retailer_good_demand
    
    num_periods = scenario_parameters.num_periods
    num_warehouses = scenario_parameters.num_warehouses
    num_goods = scenario_parameters.num_goods
    num_retailers = scenario_parameters.num_retailers
    
    warehouses = range(scenario_parameters.num_warehouses)
    goods = range(scenario_parameters.num_goods)
    retailers = range(scenario_parameters.num_retailers)
    periods = range(scenario_parameters.num_periods)
    
    if truck_rental_period == "weekly":
        num_trucks = m.addVar(lb=0, ub=max_trucks, vtype='I', name="NumTrucks")
        if branch:
            num_trucks.BranchPriority = 100
    elif truck_rental_period == "daily":
        num_trucks = m.addMVar(num_periods, lb=0, ub=max_trucks, vtype='I', name="NumTrucks")
        if branch:
            num_trucks.BranchPriority = 100
    else:
        raise NotImplementedError("Only supports `weekly` or `daily` truck rental.")
    
    truck_allocation = m.addMVar((num_warehouses, num_periods), lb=0, ub=1000, vtype='I', name="TruckAllocation")
    transport = m.addMVar((num_warehouses, num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I', name="TransportedGoods")
    carried_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods+1), lb=0, ub=10000, vtype='I', name="RetailerHeldStock")
    short_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I', name="RetailerShortStock")
    
    if truck_rental_period == "weekly":
        m.setObjective(
            gp.quicksum(transport[w, r, g, p] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods) +
            gp.quicksum(short_retailer_stock[r, g, p] * shortage_costs[g] for g in goods for r in retailers for p in periods) +
            gp.quicksum(carried_retailer_stock[r, g, p] * holding_costs[g] for g in goods for r in retailers for p in range(num_periods+1)) +
            num_trucks * truck_daily_rental_cost * num_periods,
            gp.GRB.MINIMIZE
        )
    
        # total number of trucks allocated across all warehouses for each period must equal number of trucks hired
        m.addConstrs((gp.quicksum(truck_allocation[w, p] for w in warehouses) == num_trucks for p in periods), name="TruckAllocation")
    
    elif truck_rental_period == "daily":
        m.setObjective(
            gp.quicksum(transport[w, r, g, p] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods) +
            gp.quicksum(short_retailer_stock[r, g, p] * shortage_costs[g] for g in goods for r in retailers for p in periods) +
            gp.quicksum(carried_retailer_stock[r, g, p] * holding_costs[g] for g in goods for r in retailers for p in range(num_periods+1)) +
            gp.quicksum(num_trucks[p] for p in periods) * truck_daily_rental_cost,
            gp.GRB.MINIMIZE
        )
        # total number of trucks allocated across all warehouses for each period must equal number of trucks hired
        m.addConstrs((gp.quicksum(truck_allocation[w, p] for w in warehouses) == num_trucks[p] for p in periods), name="TruckAllocation")
    else:
        raise NotImplementedError("Only supports `weekly` or `daily` truck rental.")
    
    
    # total goods sent from a warehouse in each period cannot exceed the capacity of its assigned trucks
    m.addConstrs((gp.quicksum(gp.quicksum(transport[w, r, g, p] for r in retailers) * good_sizes[g] for g in goods) <= truck_allocation[w, p] * truck_capacity for w in warehouses for p in periods), name="TruckCapacities")
    
    # first period carried retailer stock equal to 0
    m.addConstrs((carried_retailer_stock[r, g, 0] == 0 for r in retailers for g in goods), name="InitialRetailerStock")
    
    # amount of retailer goods carried each period equal to the excess demand in each period
    m.addConstrs(
        (
            carried_retailer_stock[r, g, p + 1] == gp.quicksum(transport[w, r, g, p] for w in warehouses) - retailer_good_demand[r, g, p] + carried_retailer_stock[r, g, p] + short_retailer_stock[r, g, p] for r in retailers for g in goods for p in periods
        ), name="RetailerCarryAmounts"
    )
    
    # amount of retailer goods short of demand is equal to the unmet demand in each period
    m.addConstrs(
        (
            short_retailer_stock[r, g, p] >= retailer_good_demand[r, g, p] - carried_retailer_stock[r, g, p] - gp.quicksum(transport[w, r, g, p] for w in warehouses) for r in retailers for g in goods for p in periods
        ), name="RetailerShortAmounts"
    )
    
    m.optimize()
    return m

In [4]:
def make_instance(params=None, scale=None, inst_num=None):
    np.random.seed(inst_num)
    params.num_warehouses = scale['num_warehouses']
    params.num_goods = scale['num_goods']
    params.num_retailers = scale['num_retailers']
    params.num_periods = scale['num_periods']
    
    params.good_sizes = np.random.randint(1, 20, params.num_goods)
    params.holding_costs = np.random.randint(20, 400, params.num_goods)
    params.shortage_costs = params.holding_costs * np.random.uniform(1, 1.2, size=params.num_goods)
    params.transportation_costs = np.round(np.random.uniform(1, 4, (params.num_warehouses, params.num_retailers, params.num_goods)), 1)
    params.retailer_good_demand = np.round(np.random.randint(40, 100, (params.num_retailers, params.num_goods, params.num_periods)), 1)
    return params

In [34]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 5,
    'num_goods': 30,
    'num_retailers': 15,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

In [37]:
time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)

print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

  0%|          | 0/10 [00:00<?, ?it/s]

Set parameter Cuts to value 3
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
Cuts  3

Optimize a model with 4980 rows, 16230 columns and 45505 nonzeros
Model fingerprint: 0x36ff99de
Variable types: 0 continuous, 16230 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+03, 4e+05]
  RHS range        [4e+01, 1e+02]
Using branch priorities.
Found heuristic solution: objective 9594565.6967
Presolve removed 2705 rows and 455 columns
Presolve time: 0.05s
Presolved: 2275 rows, 15775 columns, 28825 nonzeros
Variable types: 0 continuous, 15775 integer (0 binary)

Root relaxation: objective 2.760580e+05, 2457 iterations, 0.02 seconds (0.01 work u

 10%|█         | 1/10 [02:33<22:59, 153.33s/it]

Set parameter Cuts to value 3
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
Cuts  3

Optimize a model with 4980 rows, 16230 columns and 45505 nonzeros
Model fingerprint: 0x36ff99de
Variable types: 0 continuous, 16230 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+03, 4e+05]
  RHS range        [4e+01, 1e+02]
Using branch priorities.
Found heuristic solution: objective 9594565.6967
Presolve removed 2705 rows and 455 columns
Presolve time: 0.06s
Presolved: 2275 rows, 15775 columns, 28825 nonzeros
Variable types: 0 continuous, 15775 integer (0 binary)

Root relaxation: objective 2.760580e+05, 2457 iterations, 0.02 seconds (0.01 work u

 20%|██        | 2/10 [05:27<22:03, 165.38s/it]

...

 80%|████████  | 8/10 [22:33<05:46, 173.01s/it]

Set parameter Cuts to value 3
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
Cuts  3

Optimize a model with 4980 rows, 16230 columns and 45505 nonzeros
Model fingerprint: 0x36ff99de
Variable types: 0 continuous, 16230 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+03, 4e+05]
  RHS range        [4e+01, 1e+02]
Using branch priorities.
Found heuristic solution: objective 9594565.6967
Presolve removed 2705 rows and 455 columns
Presolve time: 0.06s
Presolved: 2275 rows, 15775 columns, 28825 nonzeros
Variable types: 0 continuous, 15775 integer (0 binary)

Root relaxation: objective 2.760580e+05, 2457 iterations, 0.02 seconds (0.01 work u

 90%|█████████ | 9/10 [25:28<02:53, 173.74s/it]

Set parameter Cuts to value 3
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
Cuts  3

Optimize a model with 4980 rows, 16230 columns and 45505 nonzeros
Model fingerprint: 0x36ff99de
Variable types: 0 continuous, 16230 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+03, 4e+05]
  RHS range        [4e+01, 1e+02]
Using branch priorities.
Found heuristic solution: objective 9594565.6967
Presolve removed 2705 rows and 455 columns
Presolve time: 0.06s
Presolved: 2275 rows, 15775 columns, 28825 nonzeros
Variable types: 0 continuous, 15775 integer (0 binary)

Root relaxation: objective 2.760580e+05, 2457 iterations, 0.02 seconds (0.01 work u

100%|██████████| 10/10 [28:27<00:00, 170.78s/it]


In [45]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 5,
    'num_goods': 35,
    'num_retailers': 15,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

 10%|█         | 1/10 [03:57<35:35, 237.30s/it]

--------------------------------
improved time 100.14
improved gap 0.0
benchmark time 100.12
benchmark gap 0.0


 20%|██        | 2/10 [07:58<31:54, 239.33s/it]

--------------------------------
improved time 100.14
improved gap 0.0
benchmark time 100.14
benchmark gap 0.0


 20%|██        | 2/10 [08:00<32:03, 240.44s/it]

KeyboardInterrupt



In [ ]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 5,
    'num_goods': 25,
    'num_retailers': 15,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

 10%|█         | 1/10 [02:21<21:14, 141.57s/it]

--------------------------------
improved time 37.01
improved gap 0.0
benchmark time 76.92
benchmark gap 0.0


In [5]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 5,
    'num_goods': 25,
    'num_retailers': 15,
    'num_periods': 10,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

 10%|█         | 1/10 [04:10<37:37, 250.86s/it]

--------------------------------
improved time 100.08
improved gap 0.0
benchmark time 100.21
benchmark gap 0.0


 10%|█         | 1/10 [04:19<38:56, 259.59s/it]

KeyboardInterrupt



In [ ]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 10,
    'num_goods': 25,
    'num_retailers': 25,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

 10%|█         | 1/10 [04:30<40:31, 270.14s/it]

--------------------------------
improved time 100.24
improved gap 0.0
benchmark time 100.07
benchmark gap 0.0


In [ ]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 20,
    'num_goods': 60,
    'num_retailers': 50,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

In [ ]:
params = Params()

with open("../data/daily_trucks_high_cost.json", 'r') as f:
        input_data = json.load(f)

scale = {
    'num_warehouses': 100,
    'num_goods': 500,
    'num_retailers': 200,
    'num_periods': 5,
}
params = make_instance(params, scale, inst_num=10)

params.max_trucks = int(1.2 * params.good_sizes @ np.sum(params.retailer_good_demand, axis=(0, 2)) / params.num_periods)
params.truck_capacity = input_data['truck_capacity']
params.truck_daily_rental_cost = input_data['truck_daily_rental_cost']
params.truck_rental_period = input_data['truck_rental_period']

time_improved = []
gap_improved = []
time_benchmark = []
gap_improved = []

for i in tqdm(range(10)):
    model = solve_opt_model(params, opt_param_dict={'Cuts': 3, 
                                        'TimeLimit': 100}, branch=True)
    time_improved.append(model.runtime)
    gap_improved.append(model.MIPGap)

    model = solve_opt_model(params, opt_param_dict={ 'TimeLimit': 100}, branch=False)
    time_benchmark.append(model.runtime)
    gap_improved.append(model.MIPGap)
    
    print('--------------------------------')
    print('improved time', np.round(np.mean(time_improved), 2))
    print('improved gap', np.round(np.mean(gap_improved), 2))
    print('benchmark time', np.round(np.mean(time_benchmark), 2))
    print('benchmark gap', np.round(np.mean(gap_improved), 2))

print('************************** final **************************')
print('improved time', np.round(np.mean(time_improved), 2))
print('improved gap', np.round(np.mean(gap_improved), 2))
print('benchmark time', np.round(np.mean(time_benchmark), 2))
print('benchmark gap', np.round(np.mean(gap_improved), 2))

  0%|          | 0/10 [00:00<?, ?it/s]